In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/edsa-recommender-system-predict/movies.csv
/kaggle/input/edsa-recommender-system-predict/imdb_data.csv
/kaggle/input/edsa-recommender-system-predict/genome_scores.csv
/kaggle/input/edsa-recommender-system-predict/sample_submission.csv
/kaggle/input/edsa-recommender-system-predict/tags.csv
/kaggle/input/edsa-recommender-system-predict/test.csv
/kaggle/input/edsa-recommender-system-predict/links.csv
/kaggle/input/edsa-recommender-system-predict/genome_tags.csv
/kaggle/input/edsa-recommender-system-predict/train.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/train.csv')
df_test = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/test.csv')
df_imdb = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/imdb_data.csv')
df_tags = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/tags.csv')
df_genome_tags = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/genome_tags.csv')
df_genome_scores = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/genome_scores.csv')
df_movies = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/movies.csv')
df_links = pd.read_csv('/kaggle/input/edsa-recommender-system-predict/links.csv')

# Hybrid Recommender System Metric Calculation

In [3]:
df_movies = df_movies[:20000]
#movies =  movies.reset_index(drop=True)
df_train = df_train.sample(frac=0.05)
df_train =  df_train.reset_index(drop=True)

In [4]:
from surprise.model_selection import cross_validate
df_movies.genres = df_movies.genres.str.replace('|', ' ')
#movies = movies1# (movies1, ['genres'])  
from surprise import Reader, Dataset, SVD
from sklearn.metrics.pairwise import cosine_similarity

df_movies['year'] = df_movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
df_movies['year'] = df_movies.year.str.extract('(\d\d\d\d)',expand=False)

'''Applying the Cotent_Based Filtering'''
 #Applying Feature extraction 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(stop_words='english')
#matrix after applying the tfidf
matrix=tfidf.fit_transform(df_movies['genres'])


#Compute the cosine similarity of every genre
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim=cosine_similarity(matrix,matrix)

'''Applying the Collaborative Filtering'''
#Intialising the Reader which is used to parse the file containing the ratings 
reader=Reader()

#Making the dataset containing the column as userid itemid ratings
#the order is very specific and we have to follow the same order
rating_dataset = Dataset.load_from_df(df_train[['userId','movieId','rating']],reader)
#Using the split function to perform cross validation 
#rating_dataset.split(n_folds=6)
#Intialising the SVD model and specifying the number of latent features
#we can tune this parameters according to our requirement
svd=SVD(n_factors=25)

#making the dataset to train our model
training = rating_dataset.build_full_trainset()
#training our model
svd.fit(training)
# Run 5-fold cross-validation and print results.
cross_validate(svd, rating_dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9197  0.9262  0.9224  0.9257  0.9218  0.9231  0.0024  
MAE (testset)     0.7089  0.7134  0.7096  0.7130  0.7103  0.7110  0.0018  
Fit time          21.99   22.14   22.29   22.38   22.22   22.20   0.13    
Test time         1.89    1.76    1.80    1.37    1.78    1.72    0.18    


{'test_rmse': array([0.9197404 , 0.92620665, 0.92236781, 0.92566755, 0.92175672]),
 'test_mae': array([0.7088666 , 0.7134349 , 0.70959516, 0.71302097, 0.71028207]),
 'fit_time': (21.985774278640747,
  22.144447088241577,
  22.28642964363098,
  22.381107091903687,
  22.221081733703613),
 'test_time': (1.8850903511047363,
  1.762078046798706,
  1.8049328327178955,
  1.3741581439971924,
  1.7828845977783203)}

In [5]:
#predicted_ratings=round(svd.predict(movie_id[id]).est,2)

In [6]:
from surprise.model_selection import train_test_split
from surprise import BaselineOnly
from surprise import accuracy
reader = Reader(rating_scale=(0.5,5.0))

data = Dataset.load_from_df(df_train[['userId', 'movieId', 'rating']], reader)
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.5)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
# Log RMSE

#trainset, testset = train_test_split(data, test_size=0.25)
#predictions = algo.fit(trainset).test(testset)
#accuracy.rmse(predictions)

RMSE: 0.9437


0.9437112420141897